In [4]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay):
    alpha = overlay[:, :, 3] # 투명도
    
    mask_img = alpha / 255 # 0 ~ 1 (0: 투명, 1: 불투명)
    
    for c in range(0, 3):
        image[y - h:y + h, x - w:x + w, c] = ((1 - mask_img) * image[y - h:y + h, x - w:x + w, c]) + (mask_img * overlay[:, :, c])

mp_face_detection = mp.solutions.face_detection # 얼굴 인식
mp_drawing = mp.solutions.drawing_utils # 그리기

cap = cv2.VideoCapture('face_video.mp4')

image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED)
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED)

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        ret, image = cap.read()
        
        if not ret:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose = keypoints[2]
                
                h, w, _ = image.shape
                
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose = (int(nose.x * w), int(nose.y * h))
                
                # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)
                # cv2.circle(image, left_eye, 50, (0, 0, 255), 10, cv2.LINE_AA)
                # cv2.circle(image, nose, 50, (0, 255, 0), 10, cv2.LINE_AA)
                
                # image[right_eye[1] - 50:right_eye[1] + 50, right_eye[0] - 50:right_eye[0] + 50] = image_right_eye
                # image[left_eye[1] - 50:left_eye[1] + 50, left_eye[0] - 50:left_eye[0] + 50] = image_left_eye
                # image[nose[1] - 50:nose[1] + 50, nose[0] - 150:nose[0] + 150] = image_nose      
                
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose, 150, 50, image_nose)      
            
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
      
cap.release()
cv2.destroyAllWindows()